In [ ]:
!pip install accelerate --quiet
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface_key")

from huggingface_hub import login
login(token=secret_value_0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.8 MB/s eta 0:00:00:00:0100:01


In [2]:
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests
import torch

model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [29]:
recipes = {
    "noodly tangle": {
        "file": "Classic Macaroni Salad Recipe with Video.jpg",
        "instruction": (
            "• **Cook Pasta**: Boil elbow macaroni in lightly salted water until tender (~8 min), rinse under cold water, and drain.\n"
            "• **Mix Dressing**: Whisk together mayonnaise, sugar, vinegar, mustard, salt, and pepper; toss with pasta.\n"
            "• **Combine & Chill**: Stir in celery, onion, green pepper, carrot, and pimentos. Refrigerate at least 4 hours or overnight."
        )
    },
    "creamy pink layer": {
        "file": "Judys Strawberry Pretzel Salad Recipe.jpg",
        "instruction": (
            "• **Bake Crust**: Preheat oven to 350°F. Mix crushed pretzels, melted butter, and sugar; press into a 9×13-inch pan and bake 10 min.\n"
            "• **Add Filling**: Beat cream cheese with sugar, fold in whipped topping, spread over cooled crust, and chill 30 min.\n"
            "• **Top & Set**: Dissolve Jell-O in boiling water, stir in frozen strawberries until thawed, pour over cream cheese layer, and refrigerate at least 1 hour."
        )
    },
    "spiced shred stack": {
        "file": "Pulled Chicken Shawarma Sandwich Recipe.jpg",
        "instruction": (
            "• **Cook Chicken**: Combine shawarma spices in water with garlic and onion; simmer whole chicken for 1 hour, turning every 20 min. Cool, shred meat, reserve broth.\n"
            "• **Reduce Broth**: Add bones back with water, simmer 1–2 hours, strain, and reduce broth by half.\n"
            "• **Finish & Serve**: Return shredded chicken to broth, cook until saucy, season with lemon juice and extra spices. Serve in bread or wraps."
        )
    },
    "veggie garden layers": {
        "file": "Veggie Pizza Recipe.jpg",
        "instruction": (
            "• **Bake Base**: Preheat oven to 350°F, spray a 15×10-inch pan, press crescent roll dough into pan, bake 10 min, and cool.\n"
            "• **Spread Cream**: Whip sour cream, cream cheese, ranch mix, dill, and garlic salt; spread over crust.\n"
            "• **Top & Chill**: Arrange broccoli, radish, onion, bell pepper, carrot, and celery; chill 1–2 hours, then cut into squares."
        )
    },
    "charred sea pops": {
        "file": "Spicy Grilled Shrimp Recipe.jpg",
        "instruction": (
            "• **Make Paste**: Preheat grill to medium. Crush garlic and salt, stir in paprika, cayenne, olive oil, and lemon juice to form a paste.\n"
            "• **Grill Shrimp**: Toss shrimp with paste. Oil grill grate lightly, grill shrimp 2–3 min per side until opaque.\n"
            "• **Serve Fresh**: Serve with lemon wedges."
        )
    },
    "folded veggie bundle": {
        "file": "California Club Chicken Wraps Recipe.jpg",
        "instruction": (
            "• **Make Spread**: Whisk mayonnaise, yogurt, and chipotle chiles. Microwave tortillas for 30 sec to warm.\n"
            "• **Assemble Wrap**: Spread chipotle mayo down center, top with lettuce, cheese, avocado, bacon, onion, tomato, and chicken.\n"
            "• **Roll & Serve**: Fold opposing edges over filling and roll into a tight wrap."
        )
    },
    "layered olive loaf": {
        "file": "Real Nawlins Muffuletta Recipe.jpg",
        "instruction": (
            "• **Prep Salad**: Chop olives, pepperoncini, cauliflower, cocktail onions, garlic, capers, celery, and carrot; mix with oregano, basil, pepper, celery seed, oils, and vinegar. Refrigerate covered 8 hours or overnight.\n"
            "• **Assemble Loaf**: Halve Italian loaves, hollow centers, spread olive salad on both halves.\n"
            "• **Layer & Slice**: Layer salami, ham, mortadella, mozzarella, and provolone; reassemble loaf and slice into quarters."
        )
    },
    "poppy crunch tube": {
        "file": "Chicago-Style Hot Dog Recipe.jpg",
        "instruction": (
            "• **Cook Dog**: Poach hot dog in simmering water 5 min, steam poppy-seed bun 2 min.\n"
            "• **Add Toppings**: Place dog in bun, layer mustard, relish, onion, tomato wedges, pickle spear, sport peppers.\n"
            "• **Finish Up**: Sprinkle celery salt on top—no ketchup."
        )
    },
    "spiced beef bun": {
        "file": "Tex-Mex Burger with Cajun Mayo Recipe.jpg",
        "instruction": (
            "• **Prep Grill & Sauce**: Preheat grill to medium-high, oil grate. Mix mayo with 1 tsp Cajun seasoning; set aside.\n"
            "• **Make Patties**: Combine beef, onion, 3 tsp seasoning, jalapeño, garlic, and Worcestershire; form patties.\n"
            "• **Grill & Build**: Grill 5 min per side, top with cheese in last 2 min. Spread Cajun mayo on buns, add burger, lettuce, and tomato."
        )
    },
    "cheesy wheel": {
        "file": "Campfire Pepperoni Pizza Recipe.jpg",
        "instruction": (
            "• **Prep Fire & Dough**: Place pizza stone on grill over wood fire. Roll dough to thickness.\n"
            "• **Cook Pizza**: Cook one side 10 min, flip, spread sauce, cheese, and pepperoni. Tent with foil and cook until cheese melts (~10 min).\n"
            "• **Serve Slices**: Cool slightly, slice, and serve."
        )
    }
}


image_dir = '/kaggle/input/recipie-high-res/recipes_images'
messages = []

messages.append({
    "role": "system",
    "content": [{
        "type": "text",
        "text": "You are a multimodal AI assistant specialized in cooking and recipe understanding. Your task is to generate concise cooking instructions (2–3 steps) from a food image and a noisy or vague dish title. Use your understanding of food preparation, ingredients, and cooking processes to infer what the dish likely is and how it is made, even when the input title is unclear or imprecise. Your response must be in point form, with each step beginning with a single-word heading (e.g., Chop, Mix, Fry), followed by a short, clear instruction. Use natural, human-like language and mimic the style of human-written summaries using few-shot examples provided. Ensure the instructions are practical, safe, and accurate for preparing the inferred dish. Prioritize clarity, brevity, and relevance. Do not include exotic or unsafe steps."
    }]
})

for vague_title, data in recipes.items():
    img = Image.open(f"{image_dir}/{data['file']}")
    messages.append({
        "role": "user",
        "content": [
            {"type": "image", "image": img},
            {"type": "text", "text": f'Vague Title: "{vague_title}"\nInstruction:'}
        ]
    })
    messages.append({
        "role": "assistant",
        "content": [
            {"type": "text", "text": data['instruction']}
        ]
    })

# Final Prompt Structure

In [30]:
messages

[{'role': 'system',
  'content': [{'type': 'text',
    'text': 'You are a multimodal AI assistant specialized in cooking and recipe understanding. Your task is to generate concise cooking instructions (2–3 steps) from a food image and a noisy or vague dish title. Use your understanding of food preparation, ingredients, and cooking processes to infer what the dish likely is and how it is made, even when the input title is unclear or imprecise. Your response must be in point form, with each step beginning with a single-word heading (e.g., Chop, Mix, Fry), followed by a short, clear instruction. Use natural, human-like language and mimic the style of human-written summaries using few-shot examples provided. Ensure the instructions are practical, safe, and accurate for preparing the inferred dish. Prioritize clarity, brevity, and relevance. Do not include exotic or unsafe steps.'}]},
 {'role': 'user',
  'content': [{'type': 'image',
    'image': <PIL.JpegImagePlugin.JpegImageFile image mod

# Example 1

In [31]:
test_image_path = f"{image_dir}/Dads Creamy Cucumber Salad Recipe.jpg"
test_vague_title = "chilled green slices"
messages_curr = messages.copy()
test_img = Image.open(test_image_path)
messages_curr.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [32]:
inputs = processor.apply_chat_template(
    messages_curr, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=45)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

• **Slice Cucumbers**: Slice cucumbers into ¼-inch thick rounds.
• **Make Dressing**: Whisk together yogurt, lemon juice, garlic, salt, and pepper.
• **Combine & Chill**: Toss cucumbers with dressing, garnish with fresh dill, and chill for at least 30 minutes before serving.


# Example 2

In [33]:
test_image_path = f"{image_dir}/Bang Bang Chicken Casserole Recipe.jpg"
test_vague_title = "saucy rice bed"
messages_curr = messages.copy()
test_img = Image.open(test_image_path)
messages_curr.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [34]:
inputs = processor.apply_chat_template(
    messages_curr, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=45)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

• **Cook Chicken**: Sear chicken in skillet until browned, remove and shred.
• **Combine**: Combine rice, shredded chicken, and sauce in skillet, heat through.
• **Top & Serve**: Top with cheese and sesame seeds, bake 15 minutes, serve warm.


# Example 3

In [35]:
test_image_path = f"{image_dir}/Strawberry Rhubarb Crumble Recipe.jpg"
test_vague_title = "tart berry crumble"
messages_curr = messages.copy()
test_img = Image.open(test_image_path)
messages_curr.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [36]:
inputs = processor.apply_chat_template(
    messages_curr, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=45)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

• **Prepare Fruit**: Toss sliced berries with sugar, lemon juice, and cornstarch.
• **Make Crumble**: Combine flour, oats, brown sugar, butter, and spices to form a crumble topping.
• **Bake & Serve**: Pour fruit into baking dish, top with crumble, bake at 375°F (190°C) 30-40 minutes, and serve warm with ice cream.


# Live Example 1

In [41]:
test_image_path = f"{image_dir}/Chef Johns Nashville Hot Chicken Recipe.jpg"
test_vague_title = "fiery crunch pieces"
messages_curr = messages.copy()
test_img = Image.open(test_image_path)
messages_curr.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [42]:
inputs = processor.apply_chat_template(
    messages_curr, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=45)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

• **Prepare Brine**: Soak chicken pieces in buttermilk and hot sauce for 30 min.
• **Coat & Fry**: Coat chicken in cornstarch mixture, fry in hot oil 6-8 min per side until golden brown and crispy.
• **Serve**: Top with pickles and drizzle with hot sauce.


# Live Example 2

In [43]:
test_image_path = f"{image_dir}/Classic Macaroni Salad Recipe with Video.jpg"
test_vague_title = "noodly tangle"
messages_curr = messages.copy()
test_img = Image.open(test_image_path)
messages_curr.append({
    "role": "user",
    "content": [
        {"type": "image", "image": test_img},
        {"type": "text", "text": f'Vague Title: "{test_vague_title}"\nInstruction:'}
    ]
})

In [44]:
inputs = processor.apply_chat_template(
    messages_curr, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=1000, do_sample=True, top_p=0.8, top_k=45)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

• **Cook Pasta**: Boil elbow macaroni in lightly salted water until tender (~8 min), rinse under cold water, and drain.
• **Mix Dressing**: Whisk together mayonnaise, sugar, vinegar, mustard, salt, and pepper; toss with pasta.
• **Combine & Chill**: Stir in celery, onion, green pepper, carrot, and pimentos. Refrigerate at least 4 hours or overnight.
